<h1 style="text-align: center;"><b>NetCos Pipeline</b></h1>

Starting point input for this pipeline: Fold Change data for a given disease and a group of drugs.

After running cell below, all sections can be run independently



In [1]:
import os
if not os.getcwd().endswith('src'):
    os.chdir('../src')

## Table of Contents

- [Disease data](#disease-data)
- [Filter genes by gene ID and prepare fold change data output into MITHrIL input](#filter-genes-by-gene-id-and-prepare-fold-change-data-output-into-mithril-input)
    - [find duplicated IDs](#find-duplicated-ids)
    - [Create MITHrIL input with unique gene IDs](#create-mithril-input-with-unique-gene-ids)
    - [Convert original signature data to unique gene IDs](#convert-original-signature-data-to-unique-gene-ids)
- [Run MITHrIL](#run-mithril)
    - [Run MITHrIL from command line:](#run-mithril-from-command-line)
    - [Run directly from this ipython notebook:](#run-directly-from-this-ipython-notebook)
- [Convert MITHrIL perturbation output into connectivity input](#convert-mithril-perturbation-output-into-connectivity-input)
- [Drug data](#drug-data)
- [MITHrIL input creation](#mithril-input-creation)
    - [Transpose data from gene wise to drug wise:](#transpose-data-from-gene-wise-to-drug-wise)
      - [Iteratively transpose all data](#iteratively-transpose-all-data)
      - [Transpose all data in parallel:](#transpose-all-data-in-parallel)
    - [Merge all drug data into one matrix](#merge-all-drug-data-into-one-matrix)
    - [Filter data by gene ID](#filter-data-by-gene-id)
- [MITHrIL batch run](#mithril-batch-run)
    - [Run MITHrIL from command line:](#run-mithril-from-command-line)
    - [Run MITHrIL from this notebook:](#run-mithril-from-this-notebook)
- [Convert drug mithril output into connectivity score calculation input](#convert-drug-mithril-output-into-connectivity-score-calculation-input)
    - [Iterative run:](#iterative-run)
    - [Parallel run:](#parallel-run)
  - [ADDITIONAL STEP to filter data inside metanalysis_drug_wise, because they are unfortunately still DEG data with non unique gnee filters, run the dedicated section inside preproc_utils.py, create a folder called metanalysis_drug_wise_filtered, and then zip the folder metanalysis_drug_wise.](#additional-step-to-filter-data-inside-metanalysis_drug_wise-because-they-are-unfortunately-still-deg-data-with-non-unique-gnee-filters-run-the-dedicated-section-inside-preproc_utilspy-create-a-folder-called-metanalysis_drug_wise_filtered-and-then-zip-the-folder-metanalysis_drug_wise)
  - [e anche 7_filter blalablabla è fatto allo stesso modo se si volesse usare il DEG](#e-anche-7_filter-blalablabla-è-fatto-allo-stesso-modo-se-si-volesse-usare-il-deg)
- [Calculate connectivity score](#calculate-connectivity-score)
- [Iterative run of connectivity score for a list of drugs](#iterative-run-of-connectivity-score-for-a-list-of-drugs)
- [CPU parallel run of connectivity score for a list of drugs](#cpu-parallel-run-of-connectivity-score-for-a-list-of-drugs)

# <div align="center"><b>Disease data</b>

## <b>Filter genes by gene ID and prepare fold change data output into MITHrIL input</b>

source code: step1_disease_gene_name_to_id_map.py



In [4]:
import pandas as pd
import pickle
import numpy as np
import collections
from conf import DISEASE, MITH_IN_DISEASE, TSR_OUT_DISEASE, DICT_DIR, alias_2geneid

### find duplicated IDs 

In [7]:
print('Disease symbol:', DISEASE)
disease_gene_signature_data=pd.read_csv(TSR_OUT_DISEASE+DISEASE+os.sep+DISEASE+'_signature.csv', sep=';', decimal=',',header=0)
genes_not_in_alias=[]
disease_genes_id_to_symbol_univocous={}
duplicated_ids_with_different_symbols=collections.defaultdict(list)
for symbol in disease_gene_signature_data['gene']:
    try:
        gene_id=str(alias_2geneid[symbol])
        disease_genes_id_to_symbol_univocous[gene_id] = symbol 
        duplicated_ids_with_different_symbols[gene_id].append(symbol) 
    except:

        genes_not_in_alias.append(symbol)
print(len(genes_not_in_alias), 'gene symbols without gene id ') 

#only keep duplicates:
topop=[]
for gene_id, symbols  in duplicated_ids_with_different_symbols.items():
    if len(symbols)==1:
        topop.append(gene_id)
for gene_id in topop:
    duplicated_ids_with_different_symbols.pop(gene_id)
print(len(duplicated_ids_with_different_symbols.keys()), 'duplicated ids')

Disease symbol: ipf
183 gene symbols without gene id 
530 duplicated ids


Do duplicates have different data?

In [8]:
def print_duplicated(genes,df):
    FCs=[]
    for gene_symbol in genes:
        fc=disease_gene_signature_data[disease_gene_signature_data['gene']==gene_symbol]['DE_log2_FC'].iloc[0]
        FCs.append(np.round(fc,3))
    if not len(np.unique(np.array(FCs)))==1:

        return 1
    return 0

count_duplicated_genes_with_different_FC=0
for gene_id, duped_symbols in duplicated_ids_with_different_symbols.items():
    count_duplicated_genes_with_different_FC+=print_duplicated(duped_symbols, disease_gene_signature_data)
print('tot duplicated genes with different FC', count_duplicated_genes_with_different_FC)
genes_id_to_symbol_univocous={gene_id:disease_genes_id_to_symbol_univocous[gene_id] for gene_id in topop}

tot duplicated genes with different FC 528


In [4]:
#%%save dictionary for later translation into gene names if needed:
filename=DICT_DIR+DISEASE+'_gene_id_to_symbol.pkl'
with open(filename, 'wb') as f:
    pickle.dump(genes_id_to_symbol_univocous, f)

### Create MITHrIL input with unique gene IDs

In [17]:
gene_id_mith_input=[]
for gene_id in topop: # slow
    fc=disease_gene_signature_data[disease_gene_signature_data['gene']==disease_genes_id_to_symbol_univocous[gene_id]]['DE_log2_FC'].iloc[0]
    gene_id_mith_input.append(gene_id+'\t'+str(fc))


#%% save mithril input
f=open(MITH_IN_DISEASE+DISEASE+'_signature_gene_id.mi','w')
f.write(('\n').join(gene_id_mith_input))
f.close()

### Convert original signature data to unique gene IDs

In [9]:
# remove gene symbols not in mapping
disease_gene_signature_data = disease_gene_signature_data[~disease_gene_signature_data.gene.isin(genes_not_in_alias)]

# Remove duplicates
disease_gene_signature_data['gene_id']=disease_gene_signature_data['gene'].apply(lambda gene : str(alias_2geneid[gene]))
disease_gene_signature_data = disease_gene_signature_data[disease_gene_signature_data.gene_id.isin(topop)]

# Save file
disease_gene_signature_data.to_csv(TSR_OUT_DISEASE+DISEASE+os.sep+DISEASE+'_signature_gene_id.csv', sep=';',decimal=',', index=None)


## <b>Run MITHrIL</b>

source code: step2_disease_run_MITHrIL.py


In [2]:
import subprocess
from conf import DISEASE, MITH_APP, MITH_IN_DISEASE, MITH_OUT_DISEASE

In [3]:
def run_mithril(DISEASE, MITH_APP, MITH_IN_DISEASE, MITH_OUT_DISEASE,
                organism='hsa_v2023_03', n_thread="30", verbose=True, printc=True):
    '''
    MITHrIL parameters:
    -organism hsa_v2023_03 organism version used in this experiment. For most 
            up-to-date version, use 'hsa'. for a list of all available
            organisms, type:
            java -jar MITH_APP organisms

    -reactome: also use reactome network
    -seed: random seed for replicability
    -customize-pathway-matrix : calcola alcone cose come mithril 2. va messo
    -inversion-factory fast-cpu : accelereate calculation with fast-cpu library
    -multiplication-factory fast-cpu : accelereate calculation with fast-cpu library
    -t : number of threads
    -o : full path of output file name
    -p : full path of perturbation output file name
    '''
    command = [
        "java", "-jar", MITH_APP,
        "mithril", "-reactome", "-p", "-customize-pathway-matrix",
        "-seed", "1234", "-inversion-factory", "fast-cpu", "-multiplication-factory", "fast-cpu",
        "-t", n_thread, "-i", "-organism", organism,
        f"{MITH_IN_DISEASE}{DISEASE}_signature_gene_id.mi",
        "-o", f"{MITH_OUT_DISEASE}{DISEASE}_mith3.output.txt",
        "-p", f"{MITH_OUT_DISEASE}{DISEASE}_mith3.perturbations.txt"
    ]

    if verbose:
        command.append("-verbose")

    if printc:
        print(' '.join(command))
        return
        
    else:
        # run MITHrIL from python
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        
        for line in process.stdout:
            print(line, end="")
        for line in process.stderr:
            print(line, end="")
        
        process.wait()
        return process.returncode

### Run MITHrIL from command line:

with

In [4]:
run_mithril(DISEASE, MITH_APP, MITH_IN_DISEASE, MITH_OUT_DISEASE, printc=True)

java -jar /home/signorini/mithril3/app-3.0.0-SNAPSHOT.jar mithril -reactome -p -customize-pathway-matrix -seed 1234 -inversion-factory fast-cpu -multiplication-factory fast-cpu -t 30 -i -organism hsa_v2023_03 G:\Il mio Drive\unict 2024-25\drug_repurposing\MITHrIL\input\disease_signature\ipf_signature_gene_id.mi -o G:\Il mio Drive\unict 2024-25\drug_repurposing\MITHrIL\output\disease_signature\ipf_mith3.output.txt -p G:\Il mio Drive\unict 2024-25\drug_repurposing\MITHrIL\output\disease_signature\ipf_mith3.perturbations.txt -verbose


alternatively:

### Run directly from this ipython notebook:
WARNING: running from notebook will not print any output until completion, which may take a long time. It is recomended to run from command line

In [ ]:
run_mithril(DISEASE, MITH_APP, MITH_IN_DISEASE, MITH_OUT_DISEASE, printc=False)

##  <b>Convert MITHrIL perturbation output into connectivity input

source code: step3_mith_out_to_cs_in_disease.py

In [5]:
from conf import DISEASE, MITH_OUT_DISEASE, CS_IN_DISEASE, DISEASE 
from step3_mith_out_to_cs_in_disease import mith_to_cs_in,write_cs_input

In [6]:
print(DISEASE)
# read MITHrIL perturbation output
mith_file=MITH_OUT_DISEASE+DISEASE+'_mith3.perturbations.txt'
cs_data=mith_to_cs_in(mith_file)


ipf


In [7]:
# write CS input
cs_input_name=DISEASE+'_mith3_signature.csv'
write_cs_input(cs_data, cs_input_name, CS_IN_DISEASE)

Connectivity score disease input file for MITHrIL perturbation data written at G:\Il mio Drive\unict 2024-25\drug_repurposing\connectivity_score\input\disease_signature\ipf\ 
file name: ipf_mith3_signature.csv


To explore this dataset, check the data exploration notebook

# <div align="center"><b>Drug data</b>

## <b>MITHrIL input creation</b>

Extract FC data for all genes, for all drugs at 6h, 24h and metanalysis (6h_24h) timestep from metnalayisis gene files

Results of metanalysis calculation for drug data are  saved gene-wise, where every gene has a file containing fold change data for all drugs, for different perturbation timesteps and for metanalysis timestep. This scripts refets 

source code: step4_drug_map_metanalysis_wrapper.py, map_metanalysis_gene_wise_to_drug_wise.R


### Transpose data from gene wise to drug wise:

#### Iteratively transpose all data:
run the following r script

`Rscript map_metanalysis_gene_wise_to_drug_wise.R 1 3222` 

#### Transpose all data in parallel:

In [ ]:
import subprocess

In [ ]:
# parallel execution of 31 Rscrpits of map_metanalysis_gene_wise_to_drug_wise
# each of them iterating over 100 drugs at a time 
n=1
for i in range(30):
    string='Rscript map_metanalysis_gene_wise_to_drug_wise.R ' + str(n) + ' ' + str(n+99)
    print(string)
    subprocess.Popen(['Rscript','map_metanalysis_gene_wise_to_drug_wise.R',str(n), str(n+99)])
    n+=100
    

subprocess.Popen(['Rscript','map_metanalysis_gene_wise_to_drug_wise.R','3001','3222'])

Output:
 1) drug wise metanalysis file in .Rds format
 2) 3222*3  drug-wise MITHrIL input files \<DRUGNAME\>_\<timepoint\>.mi. These files are saved into MITHrIL/input/drug_signature/

### Merge all drug data into one matrix

source: step5_mith2_to_mith3_durg_input.py


In [2]:
import os
import pandas as pd
from conf import MITH_IN_DRUG, alias_2geneid

In [3]:
df_list=[]
for mi2 in os.listdir(MITH_IN_DRUG):
    drug_condition_name=mi2.rstrip('.mi')
    if not drug_condition_name.startswith('LINCS'):
        print(drug_condition_name)
        df=pd.read_csv(MITH_IN_DRUG+mi2,sep='\t',header=None,index_col=0, names=[drug_condition_name])
        print(df.shape)
        df_list.append(df)

matrix=pd.concat(df_list, axis=1)
print(matrix.shape)
print(matrix.columns)
print(matrix.head(3))

ibuprofen_6h
(10174, 1)
ibuprofen_24h
(10174, 1)
ibuprofen_6h_24h
(10174, 1)
4-carboxy-3-hydroxyphenylglycine-(RS)_24h
(10174, 1)
4-carboxy-3-hydroxyphenylglycine-(RS)_6h
(10174, 1)
4-carboxy-3-hydroxyphenylglycine-(RS)_6h_24h
(10174, 1)
(10174, 6)
Index(['ibuprofen_6h', 'ibuprofen_24h', 'ibuprofen_6h_24h',
       '4-carboxy-3-hydroxyphenylglycine-(RS)_24h',
       '4-carboxy-3-hydroxyphenylglycine-(RS)_6h',
       '4-carboxy-3-hydroxyphenylglycine-(RS)_6h_24h'],
      dtype='object')
        ibuprofen_6h  ibuprofen_24h  ibuprofen_6h_24h  \
A2M        -0.139099       0.098812         -0.044305   
A4GALT      0.017596       0.108390          0.057804   
AAAS        0.006481      -0.038132         -0.013903   

        4-carboxy-3-hydroxyphenylglycine-(RS)_24h  \
A2M                                     -0.047952   
A4GALT                                   0.024746   
AAAS                                     0.023565   

        4-carboxy-3-hydroxyphenylglycine-(RS)_6h  \
A2M             

### Filter data by gene ID

In [4]:
def map_name_to_id(genename):
    if genename in alias_2geneid.keys():
        return alias_2geneid[genename]
    return genename
matrix.index=matrix.reset_index()['index'].apply(lambda x : map_name_to_id(x))

In [5]:
# Save MITHrIL gene id based input file:
LINCS_metanalysis_filename_id='LINCS_metanalysis.mi'
matrix.to_csv(MITH_IN_DRUG+LINCS_metanalysis_filename_id,sep='\t', index=True,  index_label=False)
print('MITHrIL drug input saved in',MITH_IN_DRUG,'filename:',LINCS_metanalysis_filename_id)

MITHrIL drug input saved in G:\Il mio Drive\unict 2024-25\drug_repurposing\MITHrIL\input\drug_signature\ filename: LINCS_metanalysis.mi


example file output:

```
drug1      drug2      drug3  
g_id1    v1         v2         v3  
g_id2    v1         v2         v3  
```
 


notice that gene ID header column is not present

## <b>MITHrIL batch run</b>
Run MITHrIL in batch version. This is useful to run several instances of MITHrIL at the same time with one input file. See function below for a list of MITHrIL batch parameters

source code: step6_drug_run_MITHrIL.py

In [3]:
import subprocess
from conf import MITH_APP, MITH_OUT_DRUG, MITH_IN_DRUG

In [6]:
def run_mithril_batch(mith_input_file, MITH_APP, MITH_IN_DRUG, MITH_OUT_DRUG, organism='hsa_v2023_03',
                 n_thread="30", verbose=True, printc=True):
    '''
    MITHrIL batch parameters:
    -organism: hsa_v2023_03 organism version used in this experiment. For most 
            up-to-date version, use default 'hsa'. for a list of all available
            organisms, type:
            java -jar MITH_APP organisms

    -reactome: also use reactome network
    -seed: random seed for replicability
    -p a flag (unlike non-batch version) to create perturbation file
    -customize-pathway-matrix : calcola alcone cose come mithril 2. va messo
    -inversion-factory fast-cpu : accelereate calculation with fast-cpu library
    -multiplication-factory fast-cpu : accelereate calculation with fast-cpu library
    -t : number of threads
    -o : output directory (unlike non-batch version, where is output name)'''
    
    command = [
        "java", "-jar", MITH_APP,
        "mithril-batch", "-reactome", "-p", "-customize-pathway-matrix",
        "-seed", "1234", "-inversion-factory", "fast-cpu", "-multiplication-factory", "fast-cpu",
        "-t", n_thread, "-organism", organism,
        "-i", f"{MITH_IN_DRUG}{mith_input_file}",
        "-o", f"{MITH_OUT_DRUG}",
        "-p"
    ]

    if verbose:
        command.append("-verbose")
    
    if printc:
        print(' '.join(command))
        return 
    else:
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        
        for line in process.stdout:
            print(line, end="")
        for line in process.stderr:
            print(line, end="")
    
        process.wait()
    
        return process.returncode

### Run MITHrIL from command line:

In [7]:
mith_input_file='LINCS_metanalysis.mi'
run_mithril_batch(mith_input_file, MITH_APP, MITH_IN_DRUG, MITH_OUT_DRUG, n_thread="30", printc=True)


java -jar /home/signorini/mithril3/app-3.0.0-SNAPSHOT.jar mithril-batch -reactome -p -customize-pathway-matrix -seed 1234 -inversion-factory fast-cpu -multiplication-factory fast-cpu -t 30 -organism hsa_v2023_03 -i G:\Il mio Drive\unict 2024-25\drug_repurposing\MITHrIL\input\drug_signature\LINCS_metanalysis.mi  -o G:\Il mio Drive\unict 2024-25\drug_repurposing\MITHrIL\output\drug_signature\ -p -verbose


### Run MITHrIL from this notebook:
WARNING: running from notebook will not print any output until completion, which may take a long time. It is recomended to run from command line


In [ ]:
run_mithril_batch(MITH_APP, MITH_IN_DRUG, MITH_OUT_DRUG, n_thread="30", printc=False)

## <b>Convert drug mithril output into connectivity score calculation input</b>

Convert the MITHrIL perturbation output into connectivity score signature inputs.

WARNING: Slow step. Parallelization recomended


source code: step7_drug_mith_out_to_cs_in.py


In [7]:
from joblib import Parallel, delayed
from preprocessing_utils import get_drugs_list, get_chunk_indexes
from step7_drug_mith_out_to_cs_in import mith_out_to_cs_in


drugs_list=get_drugs_list()
print(len(drugs_list))

2


### <b> Optional: check for file name inconsistencies between MITHrIL input and output
MITHrIL re-writes all file names that contain characters which are not
alphanumeric, '_', or '-', for safety reasons.

Run this to recuperate original file names, if needed:


In [ ]:
#Renames mith3 output files by recovering oringinal characters from input files
import os
if not os.getcwd().endswith('modules'):
    os.chdir('modules')
import pickle
import re
from conf import MITH_IN_DRUG, MITH_OUT_DRUG


#%% check diff between mith3 input and putout filenames
print('mith3 batch output filename bug: some filenames with special characters\
      differ from input to output')

mith_in_drug_names=set([n.split('.')[0] for n in os.listdir(MITH_IN_DRUG)])

def mithril_string_replace(input_string):
    pattern = r'[^a-zA-Z0-9_-]'#r'[^a-zA-Z0-9-]'
    
    # Replace matching characters with a dash
    replaced_string = re.sub(pattern, '-', input_string)
    
    return replaced_string

# print('does not work with this drug, but ok', mithril_string_replace('5\'-guanidinonaltrindole_24h'))

bad_to_good={mithril_string_replace(good):good for good in mith_in_drug_names}

for filename in os.listdir(MITH_OUT_DRUG):
    for bad_name, good_name in bad_to_good.items():
        if bad_name in filename:
            good_filename=filename.replace(bad_name, good_name)
            os.rename(MITH_OUT_DRUG+filename,MITH_OUT_DRUG+good_filename)
            print(filename,'changed to', good_filename, 'after')
            break

#%% 
print('performing final check. Anything printed is not found in output directory.\n should only print LINCS files:\n')
mith_in_drug_names=[n.split('.')[0] for n in os.listdir(MITH_IN_DRUG)]
mith_out_drug_names=[n.split('.')[0] for n in os.listdir(MITH_OUT_DRUG)]

for infile in mith_in_drug_names:
    if not infile in mith_out_drug_names:
        print(infile)

### Iterative run:

In [6]:
mith_out_to_cs_in(drugs_list, save_csv=False)


Mapping mith3 output into tsr connectivity input: 
              Removing pathway duplicates, Merging three timepoint datasets in a single file and filtering 
ibuprofen 2 of 2
drug ibuprofen processed in 9.5
writing mit3 connectivity input metanalysis files for drug ibuprofen
time elapsed: 9.583737850189209


### Parallel run:

In [8]:
# set n of cores
n_jobs=2
if n_jobs>len(drugs_list):
    raise ValueError('n_jobs > len(drugs_list)! Reduce size of n_jobs')

# get chunk size
chunk_size=int(len(drugs_list)/n_jobs)
last_chunk_size=len(drugs_list)%n_jobs

parallel_indexes=[]
for i in range(n_jobs):
    i1,i2=get_chunk_indexes(i, chunk_size)
    parallel_indexes.append((i1,i2))

results = Parallel(n_jobs=n_jobs)(delayed(mith_out_to_cs_in)\
                             (drugs_list, i1, i2,save_csv=False)\
                        for i1,i2 in parallel_indexes)

# Run last chunk if rest of division != 0:
if last_chunk_size!=0:
    mith_out_to_cs_in(drugs_list, i1, i2,save_csv=False)

# <div align="center"><b> Optional: Filter drug fold change data for connectivity score calculation</b>
Optional step, to calculate connectivity score on FC data for drugs.
Filters FC data for unique gene filters


## ADDITIONAL STEP to filter data inside metanalysis_drug_wise, because they are unfortunately still DEG data with non unique gnee filters, run the dedicated section inside preproc_utils.py, create a folder called metanalysis_drug_wise_filtered, and then zip the folder metanalysis_drug_wise.
## e anche 7_filter blalablabla è fatto allo stesso modo se si volesse usare il DEG


# <div align="center"><b>Calculate connectivity score</b>

source code: connectivity_score.py

In [2]:
from conf import DISEASE, CS_OUT
from preprocessing_utils import get_drugs_list, get_chunk_indexes
from connectivity_score import get_common_genes, bin_chen_connectivity
from loader import load_disease_signature, load_single_drug_signature

from joblib import Parallel, delayed
import pandas as pd
import numpy as np
from scipy import stats
import time
print(DISEASE)

ipf


In [3]:
# Connectivity score functions
def run_connectivity_score(DISEASE, mith, drug, pert_time='6h_24h'):
    '''
    wrapper function to load disease and drug data (for one drug), 
    and calculate their connectivity score.
    '''
    singature_uom = 'DE_log2_FC' if not mith else 'Perturbation'
    # load disease signature:
    disease_signature=load_disease_signature(DISEASE, mith=mith)
    disease_signature=disease_signature[['gene_id', singature_uom, 'adj.p.value']]
    drug_signature=load_single_drug_signature(drug, mith=mith)
    disease_common_index, drug_common_index = get_common_genes(disease_signature, drug_signature)  
    
    # filter disease signature with common genes:
    disease_signature=disease_signature.iloc[disease_common_index].reset_index(drop=True)
    print('common disease genes', len(disease_signature))
    # load drug signature
    p_val_str= 'adj.p.value' if not pert_time=='6h_24h' else 'p.value'
    columns_of_interest=['gene_id', singature_uom+'_'+pert_time, p_val_str+'_'+pert_time]
    cscore, p_value = bin_chen_connectivity(disease_signature, drug_signature[columns_of_interest], rank_on='magnitude')
    print(cscore, p_value)
    return cscore, p_value


def run_connectivity_score_drugs_batch(DISEASE, mith, drugs_list, pert_times, i1, i2, rank_on='magnitude', save_file=False):
    '''
    wrapper function to load disease and drug data (for a  batch of drugs in drugs_list[i1:i2]), 
    and calculate their connectivity score. Also calculates Pearson correlation
    coefficient, Spearman correlation coefficient, and cosine similarity, between
    disease and drug.
    Output: a dataframe of connectivity scores and  pvalues, for all combinations of conditions
    '''
    start=time.time()
    print(    '''calculates connectivity score between a disease and all LINCS database drugs, for given drugs
        and given drug perturbation times. 
        Output: a dataframe of connectivity scores and  pvalues, for all combinations of conditions''')
    
    
    singature_uom = 'DE_log2_FC' if not mith else 'Perturbation'
    
    
    print('mith tag:', mith, '\ndisease:', DISEASE)
    
    # load disease signature:
    disease_signature=load_disease_signature(DISEASE, mith=mith)
    disease_signature=disease_signature[['gene_id', singature_uom, 'adj.p.value']]
    
    
    # Discard disease genes that are 
    # not found in drug genes
    # DO NOT simply select common genes, as this would impact
    # connectrivity calculations
        #load one drug signature (all durg signatures have the same genes in the same order)
    drug_signature=load_single_drug_signature('ibuprofen', mith=mith)
    
    disease_common_index, drug_common_index = get_common_genes(disease_signature, drug_signature)  

    
    # filter disease signature with common genes:
    # Note: indexing every time with pandas .iloc is slower than writing
    # and loading binaries for filtered dataframes, for each drug
    # however this is not relevant, as we only need to filter
    # disease data, while we can take all drug data.
    disease_signature=disease_signature.iloc[disease_common_index].reset_index(drop=True)
    print('common disease genes', len(disease_signature))
    

    
    # Initialize dataframe:
    data = []
    
    
    # Calculate connectivity score between disease and drugs:     
    for drug in drugs_list[i1:i2]:
        print(drug)
        
        # load drug signature
        
        drug_signature=load_single_drug_signature(drug, mith=mith)
        
        for pert_time in pert_times:
            print(pert_time)
            
            # Calculate connectivity score between drug and disease:
            
            # select columns of interest:
            
            p_val_str= 'adj.p.value' if not pert_time=='6h_24h' else 'p.value'
            columns_of_interest=['gene_id', singature_uom+'_'+pert_time, p_val_str+'_'+pert_time]
            cscore, p_value = bin_chen_connectivity(disease_signature, drug_signature[columns_of_interest], rank_on=rank_on)
            
            # Calculate other correlations (between common genes)
            disease_signature_signature=disease_signature[singature_uom]
            drug_signature_signature=drug_signature[singature_uom+'_'+pert_time].loc[drug_common_index]
            pearson=stats.pearsonr(disease_signature_signature, drug_signature_signature)
            spearman=stats.spearmanr(disease_signature_signature, drug_signature_signature)
            cosine_sim=np.dot(np.array(disease_signature_signature),np.array(drug_signature_signature))/(np.linalg.norm(np.array(disease_signature_signature))*np.linalg.norm(np.array(drug_signature_signature)))
            
            data.append([DISEASE, drug, pert_time, cscore, p_value, pearson[0], pearson[1], spearman[0], spearman[1], cosine_sim]) # other correlations, genes subset
                
    
    connectivity_data= pd.DataFrame(data, columns=["disease","drug","perturbation_time","connectivity_score","cs_p_value",'pearson','pearson_p_value','spearman','spearman_p_value','cos_sim']) #add other correlations, genes subset
    print(connectivity_data)
    
    # save connectivity scores
    if save_file:
        if not os.path.exists(CS_OUT):
            os.mkdir(CS_OUT)
        connectivity_dataset_filename=CS_OUT+str(i1)+'_'+str(i2)+'_DEG_connectivity_score.tsv' if not mith else  CS_OUT+str(i1)+'_'+str(i2)+'_mith_connectivity_score.tsv'
        connectivity_data.to_csv(connectivity_dataset_filename, sep='\t', index=False)
    
    print('total elapsed time for', len(drugs_list[i1:i2]),' drugs: ', time.time()-start)
    return connectivity_data

## <b>Iterative run of connectivity score for a list of drugs

In [4]:
# Parameters:

drugs_list=get_drugs_list()

# Flag to decide which kind of data to calculate the connectivity score on  (0 (or False):DEG data, 1 (or True):mithril data, default: 1)
# if mith=0, a connectivity score will be calculated on normal fold change data, without the propagation step. 
# This can be useful to compare results.
mith=1

# Drug data perturbation time. (a list of either of:'6h','24h', '6h_24h' default: 6h_24h: metanalysis)
# optional: can input a list of more than one drug perturbation times.
pert_times=['6h','24h','6h_24h']

In [6]:
for drug in drugs_list:
    print(drug)
    for pert_time in pert_times:
        print(pert_time)
        cscore, p_value = run_connectivity_score(DISEASE, mith,drug, pert_time)

4-carboxy-3-hydroxyphenylglycine-(RS)
6h
common disease genes 14812
-1.978546401190291 0.275
24h
common disease genes 14812
0.027882984500222086 0.51
6h_24h
common disease genes 14812
0.007309418415434088 0.73
ibuprofen
6h
common disease genes 14812
-1.9907835445592517 0.023
24h
common disease genes 14812
-1.9856725393286072 0.108
6h_24h
common disease genes 14812
-0.00925288736031904 0.654


## <b>CPU parallel run of connectivity score for a list of drugs

In [29]:
# get drugs list
drugs_list=get_drugs_list()
# set perturbation times
pert_times=['6h','24h','6h_24h']

# Set calculation for MITHrIL data
mith=1

# set n of cores for parallel execution
n_jobs= 2 #16
if n_jobs>len(drugs_list):
    raise ValueError('n_jobs > len(drugs_list)! Reduce size of n_jobs')

# get chunk size
chunk_size=int(len(drugs_list)/n_jobs)
last_chunk_size=len(drugs_list)%n_jobs

parallel_indexes=[]
for i in range(n_jobs):
    i1,i2=get_chunk_indexes(i, chunk_size)
    parallel_indexes.append((i1,i2))


print('n jobs', n_jobs)
print('len drug list', len(drugs_list))
print('chunk size',chunk_size)
print('last chunk size', last_chunk_size)
#run_connectivity_score_drugs_batch(DISEASE, mith, drugs_list, pert_times, i1, i2)
results = Parallel(n_jobs=n_jobs)(delayed(run_connectivity_score_drugs_batch)\
                             (DISEASE, mith, drugs_list, pert_times, i1, i2)\
                        for i1,i2 in parallel_indexes)

if last_chunk_size>0:
    last_batch=run_connectivity_score_drugs_batch(DISEASE, mith, drugs_list, pert_times, i2,len(drugs_list))
    results.append(last_batch)

# build dataframe    
cs_df=pd.concat(results)

# write results
if not os.path.exists(CS_OUT):
            os.mkdir(CS_OUT)
connectivity_dataset_filename=CS_OUT+'DEG_connectivity_score.tsv' if not mith else  CS_OUT+'mith_connectivity_score.tsv'

cs_df.to_csv(connectivity_dataset_filename, sep='\t', index=False)


n jobs 2
len drug list 2
chunk size 1
last chunk size 0


In [30]:
cs_df

,disease,drug,perturbation_time,connectivity_score,cs_p_value,pearson,pearson_p_value,spearman,spearman_p_value,cos_sim
0,ipf,4-carboxy-3-hydroxyphenylglycine-(RS),6h,-1.978546,0.270,0.024081,0.003380,0.072649,8.546200e-19,0.039495
1,ipf,4-carboxy-3-hydroxyphenylglycine-(RS),24h,0.027883,0.476,0.008465,0.302960,-0.041937,3.295569e-07,-0.002157
2,ipf,4-carboxy-3-hydroxyphenylglycine-(RS),6h_24h,0.007309,0.705,0.022748,0.005628,0.010434,2.041764e-01,0.025380
0,ipf,ibuprofen,6h,-1.990784,0.026,0.002870,0.726875,-0.031195,1.463699e-04,-0.002662
1,ipf,ibuprofen,24h,-1.985673,0.093,-0.025855,0.001650,-0.073964,1.998416e-19,-0.037218
2,ipf,ibuprofen,6h_24h,-0.009253,0.654,-0.019270,0.019014,-0.072559,9.430880e-19,-0.031469
